In [165]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
from torch.utils.data import Dataset, DataLoader
import string

In [166]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [167]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [168]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [169]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [170]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [171]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [172]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [173]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val data
train_data = data[:n]
val_data = data[n:]

In [174]:
def get_batch(split):
    # Generate a small batch of data with inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [175]:
import torch.nn.functional as F

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt=None):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)

        if tgt is not None:
            seq_length = tgt.size(1)
            tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(2).to(tgt.device)  # Adjusted to work with both training and generation
            nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(tgt.device)
            tgt_mask = tgt_mask & nopeak_mask
        else:
            tgt_mask = None

        return src_mask, tgt_mask



    def forward(self, src, tgt=None):
        src_mask, tgt_mask = self.generate_mask(src, tgt)

        # If tgt is None, use a dummy tensor for tgt_embedded
        if tgt is not None:
            src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
            tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
            enc_output = src_embedded
            for enc_layer in self.encoder_layers:
                enc_output = enc_layer(enc_output, src_mask)
    
            dec_output = tgt_embedded
            for dec_layer in self.decoder_layers:
                dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
    
            output = self.fc(dec_output)
            output_mod = output.view(output.size(0), output.size(1) * output.size(2))
            tgt_mod = torch.argmax(tgt, dim=1)
            loss = F.cross_entropy(output_mod , tgt_mod)
        else:
            #src_embedded = self.dropout(self.encoder_embedding(src))
            src_embedded = self.encoder_embedding(src)
            loss = None
            tgt_embedded = torch.zeros_like(src_embedded)  # Dummy tensor for generation

            enc_output = src_embedded
            for enc_layer in self.encoder_layers:
                enc_output = enc_layer(enc_output, src_mask)
    
            dec_output = tgt_embedded
            for dec_layer in self.decoder_layers:
                dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
    
            output = self.fc(dec_output)
        return output , loss


    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # Get the predictions
            logits , loss = self(idx_cond)
            # Focus only on the last time step
            logits = logits[:, -1, :]  # Becomes (B, C)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [176]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 100
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------


src_vocab_size = vocab_size
tgt_vocab_size = vocab_size
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = block_size
dropout = 0.1

torch.manual_seed(1337)


In [177]:
model = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
#src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
#tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [178]:
xb, yb = get_batch('train')
xb.shape


# The x and y shape are of shape [16 ,32] (where 16 is batch_size and 32 is block size). Its because unlike lstm where the target is the 'price' of the nth day and the input is 'price' values
# from (n-1)th to (n-sequence_length-1). Here the target is a sequence of length block_size and the input is from (n-1)th value to block_size length. For example, for block_size 2
# For the sentence 'I think tomorrow its raining'. if target is 'tomorrow its raining' the input is 'think tomorrow its'

torch.Size([16, 32])

In [179]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
eval_interval = 100
max_iters=5000
eval_iters = 200
block_size = 32
model.train()

model = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 7.7711, val loss 7.7881
step 100: train loss 7.0845, val loss 7.0761
step 200: train loss 7.0162, val loss 7.0198
step 300: train loss 7.0439, val loss 7.0376
step 400: train loss 6.9767, val loss 6.9883
step 500: train loss 6.9793, val loss 6.9937
step 600: train loss 6.9853, val loss 7.0027
step 700: train loss 6.9828, val loss 6.9958
step 800: train loss 6.9693, val loss 6.9817
step 900: train loss 6.9696, val loss 6.9842
step 1000: train loss 7.0581, val loss 7.0814
step 1100: train loss 6.9851, val loss 7.0012
step 1200: train loss 7.0368, val loss 7.0399
step 1300: train loss 7.0093, val loss 7.0465
step 1400: train loss 6.9822, val loss 6.9823
step 1500: train loss 6.9425, val loss 6.9631
step 1600: train loss 6.9636, val loss 6.9658
step 1700: train loss 6.9395, val loss 6.9637
step 1800: train loss 6.9529, val loss 6.9630
step 1900: train loss 6.9443, val loss 6.9569
step 2000: train loss 6.9342, val loss 6.9363
step 2100: train loss 6.9389, val loss 6.9490


In [181]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


RHH3A'-H'EK:K-KF,K?K!-'-3FHCD,BH-
KK3KK-KK-MHBD-C
,'KF'
FFF FKFS:-LKF'SK
K
-'D?K&A'K&K:F-!'&
KJHA'3-3''KS
'-&-
?ABF!C!',DFIK-3-'$F&&
-KKDD-.A--'-AK-'FF3K$HAKa'3J-.DFS,F-KS-iQKN:!-
KK-''i:F3LC-FJ'3ICN3AMKK3D33'-',C-
!:C
!SAD'SK-3-k,'K-KK
KK-KIK3-K3-'!
:E'K!!CK?KKD!-C-3-'DN
N'$J-DKCSc-
,KSCKF
FKK.J'-
,'F'SL3ESC:KG$3EHKKBDIK $FK3C'K,''&DQK,:-!-FD3-CKK'KK'-K
KH$w&',&FD,!M
K
M--I'K
F-&KK
-DFKC''3N!'-JK3F'C-S,K,K,K3K'JF,.!?S3K'NK,-KJSKKES,K,$R3.K',-S-3K&;:KKC3-!K,H!'
KFK!-D:K'HHQ-I.JKH$F'KG'K!-.$!CKK&AK.-$HC3!-AK:SK,!KKK:K-JK-
,,-3o-A!K'K
-3!SKK-NKCA'?-
K-K,D!:
!!KKRF-
;CS&K3-I'E':'JM$K''''3',-C-
:-EJKE&!3,GvM3P''-':3KQC-KC3O

-KC-&:S&'KF'S3'K.KNK&KKA-
V-N!:!C-C.K$S$D-3CE?N3E$H:K-'$
KKK-K'GK.'cD!'N
F'--
S
-C!3S--K3-F'SJ-K'CKGD-,LIK''I3!''GK--K,'3K !.-FSKGH,K3K-IKCjK&JJ-
K,-G'''F3!J'DKK'F-'3',K.-3$-J-Q
SSKK:KM'KEK!$DIK','!
GK
.KD:K-KKK','-E
$.'--!FK-K3SK:JNA3
K'KSK'-'HKKK!A,K
&S-':POSD33
AEK--'K3-AA!E3N3CHK-K-H-:''.!SD$KLR'?$C3EK'
SKKM:KCJS$DJ-3K'KKKF,
Hx'iK--K;NK-3SS'3--KKC-KA-K;.NK;'KK
K-L

In [ ]:
transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: 8.68002700805664
